In [1]:
%matplotlib inline

import ndex2
import math
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import spearmanr

In [2]:
# Load gene summary table# Load  
gene_summaries_given = pd.read_csv("../src/gene_level_summary_stats_pmid_25056061.txt", 
                                   header = 0, sep = '\t', index_col = 1)
gene_summaries_given.head()

,Unnamed: 0,Chr,Gene Start,Gene End,nSNPs,TopSNP,TopSNP Pos,TopSNP P-Value,SNP Distance
Gene,,,,,,,,,
HIST1H4K,5905,6,27906930,27907284,8,rs34706883,27913234.0,5.071180e-10,6304.0
HIST1H2AK,5867,6,27913636,27914096,16,rs34706883,27913234.0,5.071180e-10,402.0
HIST1H2BN,5883,6,27914418,27914867,17,rs34706883,27913234.0,5.071180e-10,1184.0
HIST1H2AL,5868,6,27941085,27941555,10,rs13199772,27942064.0,7.053790e-10,979.0
HIST1H1B,5855,6,27942548,27943338,10,rs13199772,27942064.0,7.053790e-10,484.0


In [3]:
pgc_snp_summaries = pd.read_csv("../data/pgc_test.txt", sep = '\t', header = 0, index_col=1)
pgc_snp_summaries.head()

,hg18chr,bp,a1,a2,or,se,pval,info,ngt
snpid,,,,,,,,,
rs10907175,1,1120590,A,C,1.0142,0.0354,0.691510,0.9922,13
rs2887286,1,1145994,T,C,0.9862,0.0278,0.617802,0.9989,17
rs11260562,1,1155173,A,G,0.9997,0.0466,0.995511,0.9281,11
rs7519837,1,1500664,T,C,1.0139,0.0226,0.540971,0.9819,13
rs12126768,1,1767950,T,G,1.0256,0.0270,0.348908,0.9932,5


In [4]:
# Assign SNPs to genes based on +/- nkb window
nkb = 10

# Init new gene_summaries dataframe
gene_summaries = gene_summaries_given.loc[:, ["Chr", "Gene Start", "Gene End"]]
gene_summaries.head()

,Chr,Gene Start,Gene End
Gene,,,
HIST1H4K,6,27906930,27907284
HIST1H2AK,6,27913636,27914096
HIST1H2BN,6,27914418,27914867
HIST1H2AL,6,27941085,27941555
HIST1H1B,6,27942548,27943338


In [6]:
gene_scores = []
snp_counts = []
min_p = []
for gidx, gene in gene_summaries.iterrows():
    chrom = gene["Chr"]
    
    # filter for chromosome
    this_snps = pgc_snp_summaries[pgc_snp_summaries["hg18chr"] == chrom]
    
    # filter for upstream endpoint
    this_snps = this_snps[this_snps["bp"] >= gene["Gene Start"] - (nkb * 1000)]
    
    # filter for downstream endpoint
    this_snps = this_snps[this_snps["bp"] <= gene["Gene End"] + (nkb * 1000)]
    
    # Calculate adjusted bonferroni-adjusted p-value -> "gene score"
    # pvals = [p * snp_summaries.shape[0] for p in this_snps.loc[:, "pval"].tolist()]
    pvals = [p for p in this_snps.loc[:, "pval"].tolist()]
    adj_score = min(pvals) / len(pvals)
    min_p.append(min(pvals))
    gene_scores.append(adj_score)
    snp_counts.append(len(pvals))
gene_summaries["Gene Score"] = -np.log(gene_scores)
gene_summaries["nSNPs"] = snp_counts
gene_summaries["TopSNP P-Value"] = min_p

ValueError: min() arg is an empty sequence

In [ ]:
gene_summaries.head()
